In [ ]:
# class MonteCarloTreeSearch:
#     def __init__(self, env, V, initial_state=None, max_steps=2):
#         self.env = env
#         self.V = V  # Value matrix
#         self.action_space = self.env.action_space.n
#         self.max_steps = max_steps  # Limit search depth to 2 steps
#         self.initial_state = initial_state
#         self.reset_tree()

#     def reset_tree(self):
#         """Reset the tree with the given state as the root."""
#         if self.initial_state is None:
#             state = self.env.reset()[0]
#         else:
#             self.env.reset()  # Reset the environment to the start.
#             self.env.unwrapped.s = self.initial_state  # Manually set the state to initial_state.
#             state = self.initial_state
        
#         root_node = Node(state=state, action=None, action_space=self.action_space, reward=0, terminal=False)
#         self.tree = Tree()
#         self.tree.add_node(root_node)

#     def expand(self, node):
#         action = node.untried_action()
        
#         # Save the environment's state before taking the action
#         previous_state = self.env.unwrapped.s
        
#         state, _, done, _, _ = self.env.step(action)
#         new_node = Node(state=state, action=action, action_space=self.action_space, reward=self.V[state], terminal=done)
#         self.tree.add_node(new_node, node)

#         # Restore the environment's state
#         self.env.unwrapped.s = previous_state

#         return new_node

#     def default_policy(self, node):
#         # Ensure the environment is set to the state of the node before the simulation
#         state = node.state
#         total_reward = 0
#         done = node.terminal

#         # Manually set the environment to the node's state
#         self.env.unwrapped.s = state
        
#         print(f"Starting simulation from state: {state}")

#         # First step
#         if not done:
#             action = random.choice(range(self.action_space))
#             state, reward, done, _, _ = self.env.step(action)
#             total_reward += self.V[state]
#             print(f"  Step 1 - Action: {action}, New State: {state}, Reward: {reward}, Done: {done}")

#         # Second step
#         if not done:
#             action = random.choice(range(self.action_space))
#             state, reward, done, _, _ = self.env.step(action)
#             total_reward += self.V[state]
#             print(f"  Step 2 - Action: {action}, New State: {state}, Reward: {reward}, Done: {done}")

#         print(f"Total reward from simulation: {total_reward}")
#         return total_reward


#     def tree_policy(self):
#         node = self.tree.root
#         depth = 0

#         while not node.terminal and depth < self.max_steps:
#             if self.tree.is_expandable(node):
#                 return self.expand(node)
#             else:
#                 node = random.choice(self.tree.children(node))  # Move to one of the children
#             depth += 1

#         return node

#     def backward(self, node, value):
#         while node:
#             node.num_visits += 1
#             node.total_simulation_reward += value
#             node.performance = node.total_simulation_reward / node.num_visits
#             node = self.tree.parent(node)

#     def forward(self):
#         """Perform a single iteration of MCTS with two-level deep simulation as default policy."""
#         leaf_node = self.tree_policy()
#         print(f"Selected leaf node for expansion: {leaf_node.state}")
#         simulation_result = self.default_policy(leaf_node)
#         self.backward(leaf_node, simulation_result)
        
#     def run(self):
#         """Run a single iteration, now reflecting the deeper default policy."""
#         self.forward()

#     def choose_best_action(self):
#         """After running the MCTS, choose the best action based on the highest value of the final state."""
#         best_child = max(self.tree.children(self.tree.root), key=lambda n: self.V[n.state], default=None)
#         if best_child:
#             return best_child.action, best_child.state
#         else:
#             return None, None


In [ ]:

# def main():
#     env = gym.make('FrozenLake-v1', is_slippery=False)
#     env2 = gym.make('FrozenLake-v1', is_slippery=False)

#     # Load and compute the V-table from the Q-table
#     q_table_file = 'Q.npy'  # Replace with the correct path to the Q-table file
#     v_table = compute_v_table_from_q_table(q_table_file)
    
#     print("V-table:", v_table)
    
#     # Define the target state
#     target_state = 15
    
#     # Initialize starting state
#     stateMCTS = env.reset()[0]
#     print(f"Initial state: {stateMCTS}")
#     state2 = env2.reset()[0]
    
#     path = [stateMCTS]
#     terminal_state_reached = False
#     max_iterations_without_convergence = 10000
#     iteration_count = 0

#     while not terminal_state_reached and iteration_count < max_iterations_without_convergence:
#         # Initialize Monte Carlo Tree Search with the current state
#         monteCarloTreeSearch = MonteCarloTreeSearch(env=env, V=v_table, initial_state=stateMCTS)
#         monteCarloTreeSearch.run()
#         print(f"\nBuilt tree for the cell\n")
#         monteCarloTreeSearch.tree.show()
        

#         # Get the best action from the current state
#         action, next_state = monteCarloTreeSearch.choose_best_action()

#         if action is None:
#             print("No valid actions found, stopping.")
#             break

#         print(f"Chosen action: {action}, leads to state: {next_state}")
#         path.append(next_state)

#         # Apply the action in env2
#         state2, reward, done, _, _ = env2.step(action)
#         print(f"New state after action {action}: {state2}, reward: {reward}, done: {done}")

#         # Check if the new state is the terminal state (goal) or if it is a terminal state (falling into the lake)
#         if state2 == target_state:
#             print("Target state reached.")
#             terminal_state_reached = True
#         elif done:
#             print("Fell into the lake, retrying.")
#             stateMCTS = env.reset()[0] 
#             env2.reset()  # Reset the environment and get the new initial state
#             path = [stateMCTS]  # Reset the path for new attempt
#             iteration_count = 0  # Reset the iteration count for new attempt
#         else:
#             # Update stateMCTS to continue MCTS from the new state
#             stateMCTS = state2
#             iteration_count += 1

#     # Clear path if target state is not reached
#     if not terminal_state_reached:
#         path = []

#     print("\nFinal path (states):", path)

# if __name__ == "__main__":
#     main()